In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Saura\\OneDrive\\Desktop\\Upskilling-Reskilling\\Python and ML\\Projects\\NLP_Txt_Summarizer\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Saura\\OneDrive\\Desktop\\Upskilling-Reskilling\\Python and ML\\Projects\\NLP_Txt_Summarizer\\Text-Summarizer-Project'

In [10]:
#enity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path
    



In [15]:
#Configuration Manager in src/textSummarizer/config
from textSummarizer.constants import *
from textSummarizer.utils.common import *


class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_validation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name,
        )
        return data_validation_config

In [16]:
#components
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)
        
    def convert_examples_to_features(self, example_batch):
        input_encodings=self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
       'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
        }

    def convert(self):
        dataset_samsum= load_from_disk(self.config.data_path)
        dataset_samsum_pt= dataset_samsum.map(self.convert_examples_to_features,batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))


In [17]:
#pipeline
try:
    config= ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(data_transformation_config)
    data_transformation.convert()

except Exception as e:
    logger.error(f"Error in data transformation: {str(e)}")
    raise e


[2024-07-07 20:45:48,791: INFO: common 24 yaml file : config\config.yaml loaded successfully]
[2024-07-07 20:45:48,794: INFO: common 24 yaml file : params.yaml loaded successfully]
[2024-07-07 20:45:48,795: INFO: common 46 Directory already exists : artifacts]
[2024-07-07 20:45:48,795: INFO: common 43 Directory created : artifacts\data_transformation]


c:\Users\Saura\anaconda3\envs\venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\Saura\anaconda3\envs\venv\lib\site-packages\transformers\tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 63581.86 examples/s]
